In [18]:
import pandas as pd
import numpy as np
import re


In [19]:
df= pd.read_csv('../dataset/final.csv')
df.head()

,labels,tweet
0,0.0,#AAPL:The 10 best Steve Jobs emails ever...htt...
1,0.0,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...
2,0.0,My cat only chews @apple cords. Such an #Apple...
3,0.0,I agree with @jimcramer that the #IndividualIn...
4,0.0,Nobody expects the Spanish Inquisition #AAPL


In [20]:
df.iloc[0].tweet

'#AAPL:The 10 best Steve Jobs emails ever...http://t.co/82G1kL94tx'

### Deep Data Cleaning

In [21]:
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [22]:
import string
texts_new = []
for t in df.tweet:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(t)))))

In [37]:
df['text']=texts_new
df['labels'] = [int(l) for l in df['labels']]
for i, text in enumerate(df['text']):
    if text == '':
        print(df[i])
newdf = df[['labels', 'text']].copy()

5758
5790
5809
5830
5870
5873
5899
5904
5905
5942
5971
6014
6021
6029
6050
6064
6080
6087
6106


In [15]:
text_len = []
for text in df.clean_text:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

In [27]:
newdf.to_csv('../dataset/data.csv', index=False)

In [11]:
df.iloc[0].clean_text

'aaplthe 10 best steve jobs emails ever'

In [12]:
df.head()

,sentiment,tweet,clean_text,text_len
0,0.0,#AAPL:The 10 best Steve Jobs emails ever...htt...,aaplthe 10 best steve jobs emails ever,7
1,0.0,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...,rt why aapl stock had a miniflash crash today ...,11
2,0.0,My cat only chews @apple cords. Such an #Apple...,my cat only chews cords such an applesnob,8
3,0.0,I agree with @jimcramer that the #IndividualIn...,i agree with that the individualinvestor shoul...,21
4,0.0,Nobody expects the Spanish Inquisition #AAPL,nobody expects the spanish inquisition aapl,6


### Balancing the dataset

In [13]:
df['sentiment'].value_counts()

sentiment
 0.0    3676
-1.0    2235
 1.0     704
Name: count, dtype: int64

In [68]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['clean_text']).reshape(-1, 1), np.array(df['sentiment']).reshape(-1, 1))
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['clean_text', 'sentiment'])
train_os['sentiment'].value_counts()

sentiment
 0.0    3676
 1.0    3676
-1.0    3676
Name: count, dtype: int64

In [114]:
train_os = train_os[train_os.clean_text != '']
train_os.to_csv('../dataset/data.csv', index=None)

### train-validation dataset split

In [123]:
from datasets import load_dataset

dataset = load_dataset("csv", split='train', data_files="../dataset/data.csv")
print(dataset['clean_text'][5758])

descola a ios7 logo porra


### Tokenizing

In [125]:
X_train, X_valid, y_train, y_valid = train_test_split(dataset['clean_text'], dataset['sentiment'], test_size=0.1, random_state=42)

print(X_train)

['my phone just charged 23 in 45 seconds thanks', 'wouldnt be a bad idea to bring back the tap to tweet function guys', 'rt protesters stage diein protests in store in nyc is it me or is this anger misplaced retweet if you agree', 'have patented an iphone drop protection mechanism is the is end of crackedscreens', 'tell to update their events page theres no listing for your presentation', 'apples iphone 6 plus amazingly captures 41 of the phablet market right out of the gate aapl aapl', 'is apple hedging its bets with another small iphone aapl', 'ironic that were at a pr event mp15 talking about companies that dont need more people talking about them', 'thank u i can now compile all of the pics that i communicate with in one place', 'all hyperbole aside this article as well as are simply amazing arent they via', 'what to expect in 2015', 'smh rt rt the worst rt yooo why my iphone 5 battery so sus', 'afterhours buzz aapl bobe amp more check out which companies are making headlines after

In [126]:
tokenizer= RobertaTokenizerFast.from_pretrained("roberta-base")

In [130]:
def tokenize(examples):
    token = tokenizer(examples['clean_text'], truncation=True, padding="max_length")
    return token

tokenized_dataset = dataset.map(tokenize)

In [154]:
tokenized_dataset

Dataset({
    features: ['clean_text', 'sentiment', 'input_ids', 'attention_mask'],
    num_rows: 11009
})

### RoBERTa Sentiment Analysis

### Building Model

In [148]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

config.json: 100%|██████████| 570/570 [00:00<00:00, 1.01MB/s]
model.safetensors: 100%|██████████| 436M/436M [01:07<00:00, 6.48MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [149]:
small_train_dataset = tokenized_dataset.shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_dataset

In [156]:
from transformers import TrainingArguments
training_args = TrainingArguments("test_trainer")
from transformers import Trainer
trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

## PPLX rocks


In [79]:

# Freeze pre-trained layers
for layer in roberta_model.layers:
    print(layer)
    layer.trainable = False

print(roberta_model.summary())
# Add new layers for your specific task

# print(model.summary())
# model.build(X_train.shape)

# print(model.summary())

# print(roberta_model.summary())
# Compile the model
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
# model.fit(X_train, epochs=10, validation_data=X_valid)


Model: "tf_roberta_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124055040 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592899    
 ificationHead)                                                  
                                                                 
Total params: 124647939 (475.49 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 124647939 (475.49 MB)
_________________________________________________________________
None
